In [1]:
# Importing libraries, APIS & Tensorflow

# import future

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

In [2]:
# import tensorflow & keras

import tensorflow as tf
import keras

In [3]:
# import modules, libraries, APIs, optimizers, layers, etc...

from tensorflow.keras import Sequential
from tensorflow.keras.layers import (
    Flatten,
    Dense,
    Dropout,
    BatchNormalization,
    Conv2D,
    MaxPool2D,
    MaxPool1D
)

In [4]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping

In [5]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [6]:
from keras.models import model_from_json
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.applications import VGG16, xception
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [7]:
from keras import models
from keras import layers
from keras import optimizers
import scipy
from scipy.interpolate import make_interp_spline
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sn
import PIL
from PIL import Image
from contextlib import redirect_stdout
import sys

In [10]:
# print file name

file_name =  os.path.basename(sys.argv[0])
file_name = str(file_name)

print("Name of the File:", file_name)

# Trim the ".py" extensiojn from the file 

file_name = file_name[:-3]

# set another (very high) limit for image processing

PIL.Image.MAX_IMAGE_PIXELS = 9933120000

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

Name of the File: ipykernel_launcher.py


In [32]:
# create the list where the accuracy rates of each dataset will be stored

list_acc_rates = list()

# set the number of frozen layers that will not be trained

num_freeze = 132

In [18]:
# setting the paths, pathfiles

parent_dir = "C:/Users/Teclast/Documents/Papers/Deep Learning in Artworks/Database/"
path_train_images = parent_dir
saved_model_path = "C:/Users/Teclast/Documents/Papers/Deep Learning in Artworks/Experiments/Sentiment/Sentiment_model_performance/"
path_scripts = "C:/Users/Teclast/Documents/Papers/Deep Learning in Artworks/Experiments/Sentiment/Sentiment_model_architecture/"

In [22]:
meganum = 1
num_dataset = 1

new_file_name = file_name + "_df(" + str(meganum) + ")"
print(new_file_name)

train_labels_path = parent_dir + "Labels/" + "Emotion_set_" + str(num_dataset) +".xlsx"

### Setting up all the parameters for the training, processing of data and produced files management ###

### setting parameters for the saving and loading of produced files ###

# set batch size

# num_batch = 10

# adjusting the size of images into the same dimensions

img_width = 71
img_height = 71

# adjsusting training parameters

lrn_rate = 0.0001  # learning rate
num_epochs = 5  # number of training epochs
act_func = "sigmoid"  # activation fuction

num_neurons_1st_dense = 100 # number of neurons in first dense layer
drop_perc_1st_dense = 0 # percentage of dropout rate in first dense layer

num_neurons_2nd_dense = 50 # number of neurons in second dense layer
drop_perc_2nd_dense = 0 # percentage of dropout rate in second dense layer

num_neurons_3d_dense = 0 # number of neurons in second dense layer
drop_perc_3d_dense = 0 # percentage of dropout rate in second dense layer

# import the labels of the dataset

data = pd.read_excel(train_labels_path, engine="openpyxl")

# check the dataframe to inspect any NaNs or missing values

print(data.head(3))

# check the number of rows and columns of the dataframe

print(data.shape)

ipykernel_launcher_df(1)
            Artist_Name                               File_Name    Art_Class  \
0  Alphonse Maria Mucha  alphonse-mucha_6th-sokol-festival-1912  Art Nouveau   
1  Alphonse Maria Mucha  alphonse-mucha_8th-sokol-festival-1912  Art Nouveau   
2  Alphonse Maria Mucha                      alphonse-mucha_aaw  Art Nouveau   

  Emotion_Class  Joy  Sadness  Disgust  Surprise  Anger  Fear  
0       Disgust    0        0        1         0      0     0  
1           Joy    1        0        0         0      0     0  
2           Joy    1        0        0         0      0     0  
(15292, 10)


In [23]:
X = []

# attach the images with the labels, in order to have supervised learning and convert them to have values between 0 - 1 by multiplying with 255 (RGB values: 0 - 255)

for i in tqdm(range(data.shape[0])):
    path = (
        path_train_images
        + data["Art_Class"][i]
        + "/"
        + data["Artist_Name"][i]
        + "/"
        + data["File_Name"][i]
        + ".jpg"
    )
    img = image.load_img(path, target_size=(img_width, img_height, 3))
    img = image.img_to_array(img)
    img = img / 255.0
    X.append(img)


# convert images into np array

X = np.array(X)

# get the size of dataframe X

print("The shape of X list is: ", X.shape)

100%|██████████| 15292/15292 [11:39<00:00, 21.87it/s] 


The shape of X list is:  (15292, 71, 71, 3)


In [60]:
y_starting = data

# create the train/test sets, set the size of test set

X_train, X_test, y_starting_train, y_starting_test = train_test_split(X, y_starting, test_size=0.2, random_state=1)

# create the test/validation sets, set the size of validation set

X_train, X_val, y_starting_train, y_starting_val = train_test_split(X_train, y_starting_train, test_size=0.125, random_state=1)

# removing "unnecessary columns, in order to have only the one-hot encoded columns for the training

y_train = y_starting_train.drop(["Artist_Name", "File_Name", "Art_Class", "Emotion_Class"], axis=1)
y_test = y_starting_test.drop(["Artist_Name", "File_Name", "Art_Class", "Emotion_Class"], axis=1)
y_val = y_starting_val.drop(["Artist_Name", "File_Name", "Art_Class", "Emotion_Class"], axis=1)

# set that the evaluation set is 10% of the total dataset
# the testing dataset will be 20% of the total dataset
# the final percentage of training set will be 70% of the total dataset

y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)

# get the shapes of train, test and validation splits

print("The shape of X_train is: ", X_train.shape)
print("The shape of y_train is: ", y_train.shape)

print("The shape of X_test is: ", X_test.shape)
print("The shape of y_test is: ", y_test.shape)

print("The shape of X_val is: ", X_val.shape)
print("The shape of y_val is: ", y_val.shape)

The shape of X_train is:  (10703, 71, 71, 3)
The shape of y_train is:  (10703, 6)
The shape of X_test is:  (3059, 71, 71, 3)
The shape of y_test is:  (3059, 6)
The shape of X_val is:  (1530, 71, 71, 3)
The shape of y_val is:  (1530, 6)


In [62]:
y_val[2]

array([1, 0, 0, 0, 0, 0], dtype=int64)

In [33]:
# Load the Xception model

xception_conv = xception.Xception(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# we chose blocks, we will freeze in order to adopt the train weights
# unfreeze the rest:

for layer in xception_conv.layers[:num_freeze]:
  layer.trainable = False

for layer in xception_conv.layers[num_freeze:]:
  layer.trainable = True

# Check the trainable status of the individual layers

for layer in xception_conv.layers:
    print(layer.name, layer.trainable)

# Create the model

model = models.Sequential()

# Add the Xception convolutional base model

model.add(xception_conv)

# flatten all layers

model.add(layers.Flatten())

# Add new layers

# Add first additional dense layer

model.add(layers.Dense(num_neurons_1st_dense, activation='relu'))
#model.add(layers.Dropout(drop_perc_1st_dense))

# Add second additional dense layer

model.add(layers.Dense(num_neurons_2nd_dense, activation='relu'))
#model.add(layers.Dropout(drop_perc_2nd_dense))

# Add third additional dense layer (if needed)

#model.add(layers.Dense(num_neurons_3d_dense, activation='relu'))
#model.add(layers.Dropout(drop_perc_3d_dense))

model.add(layers.Dense(6
, activation= act_func))

# Show a summary of the model. Check the number of trainable parameters

model.summary()    

input_4 False
block1_conv1 False
block1_conv1_bn False
block1_conv1_act False
block1_conv2 False
block1_conv2_bn False
block1_conv2_act False
block2_sepconv1 False
block2_sepconv1_bn False
block2_sepconv2_act False
block2_sepconv2 False
block2_sepconv2_bn False
conv2d_12 False
block2_pool False
batch_normalization_12 False
add_36 False
block3_sepconv1_act False
block3_sepconv1 False
block3_sepconv1_bn False
block3_sepconv2_act False
block3_sepconv2 False
block3_sepconv2_bn False
conv2d_13 False
block3_pool False
batch_normalization_13 False
add_37 False
block4_sepconv1_act False
block4_sepconv1 False
block4_sepconv1_bn False
block4_sepconv2_act False
block4_sepconv2 False
block4_sepconv2_bn False
conv2d_14 False
block4_pool False
batch_normalization_14 False
add_38 False
block5_sepconv1_act False
block5_sepconv1 False
block5_sepconv1_bn False
block5_sepconv2_act False
block5_sepconv2 False
block5_sepconv2_bn False
block5_sepconv3_act False
block5_sepconv3 False
block5_sepconv3_bn False

In [34]:
len(xception_conv.layers)

132

In [35]:
# Compile the model

model.compile(optimizer = RMSprop(lr=lrn_rate),
            loss = 'binary_crossentropy',
            metrics = ['acc'])


history = model.fit(X_train, y_train,
            batch_size=20,
            epochs=num_epochs,
            validation_data=(X_test, y_test))


print(history.history.keys())

Epoch 1/5
 11/536 [..............................] - ETA: 36:55 - loss: 0.5377 - acc: 0.2409

KeyboardInterrupt: 

In [39]:
X_train[0]

array([[[0.70980394, 0.627451  , 0.54509807],
        [0.7764706 , 0.69803923, 0.6039216 ],
        [0.80784315, 0.70980394, 0.6313726 ],
        ...,
        [0.62352943, 0.5372549 , 0.54509807],
        [0.5921569 , 0.5176471 , 0.53333336],
        [0.74509805, 0.6431373 , 0.6392157 ]],

       [[0.84705883, 0.78039217, 0.70980394],
        [0.79607844, 0.7294118 , 0.6666667 ],
        [0.7176471 , 0.6509804 , 0.5882353 ],
        ...,
        [0.7921569 , 0.69803923, 0.7490196 ],
        [0.67058825, 0.5764706 , 0.6313726 ],
        [0.72156864, 0.627451  , 0.6784314 ]],

       [[0.8039216 , 0.7490196 , 0.7137255 ],
        [0.80784315, 0.7411765 , 0.70980394],
        [0.77254903, 0.68235296, 0.627451  ],
        ...,
        [0.7411765 , 0.654902  , 0.6627451 ],
        [0.73333335, 0.65882355, 0.6745098 ],
        [0.7490196 , 0.6745098 , 0.69803923]],

       ...,

       [[0.69803923, 0.6862745 , 0.6509804 ],
        [0.14117648, 0.14901961, 0.06666667],
        [0.3019608 , 0

In [68]:
df_repair = pd.read_excel('C:/Users/Teclast/Desktop/Repair.xlsx',engine = 'openpyxl')
df_repair

,Artist_Name,File_Name,Art_Class,True_Emotion_Class,Predicted_Emotion_1,Predicted_Emotion_2,Predicted_Emotion_3


In [70]:
df_repair['Artist_Name'] = y_starting_val['Artist_Name']
df_repair['File_Name'] = y_starting_val['File_Name']
df_repair['Art_Class'] = y_starting_val['Art_Class']
df_repair['True_Emotion_Class'] = y_starting_val['Emotion_Class']

df_repair.head(3)

,Artist_Name,File_Name,Art_Class,True_Emotion_Class,Predicted_Emotion_1,Predicted_Emotion_2,Predicted_Emotion_3
2648,Juan Gris,juan-gris_the-violin-1914,Cubism,Sadness,NaN,NaN,NaN
8864,Gustave Loiseau,gustave-loiseau_by-the-eure-river,Post-Impressionism,Joy,NaN,NaN,NaN
14868,Salvador Dali,salvador-dali_portrait-of-colonel-jack-warner,Surrealism,Joy,NaN,NaN,NaN


In [72]:
for i in range(1,4):
    column_name = 'Predicted_Emotion_' + str(i)
    print(column_name)
    print(type(column_name))

Predicted_Emotion_1
<class 'str'>
Predicted_Emotion_2
<class 'str'>
Predicted_Emotion_3
<class 'str'>


In [74]:
df_repair[column_name]

2648     NaN
8864     NaN
14868    NaN
3994     NaN
503      NaN
        ... 
4272     NaN
154      NaN
14110    NaN
11556    NaN
1819     NaN
Name: Predicted_Emotion_3, Length: 1530, dtype: object